# COVID-19 Case Counts from Gobierno de Mexico

**[Work in progress]**

This notebook creates a .csv file with cummulative confimed cases and deaths for ingestion into the Knowledge Graph.

Data source: [COVID-19 Mexico, Gobierno de Mexico](https://coronavirus.gob.mx/datos)

Author: Peter Rose (pwrose@ucsd.edu)

In [1]:
import os
from pathlib import Path
import pandas as pd
import datetime
import dateutil

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_IMPORT = Path(os.getenv('NEO4J_IMPORT'))
print(NEO4J_IMPORT)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-328d8379-6ab4-4cc1-a397-2de37909d2e4/installation-4.1.0/import


#### Try three dates to get latest data

In [4]:
today = datetime.datetime.utcnow().date()
tomorrow = today - datetime.timedelta(days=-1)
yesterday = today - datetime.timedelta(days=1)

In [5]:
today = str(today).replace('-', '')
tomorrow = str(tomorrow).replace('-', '')
yesterday = str(yesterday).replace('-', '')

In [6]:
print('yesterday', yesterday, ', today', today, ', tomorrow', tomorrow)

yesterday 20200911 , today 20200912 , tomorrow 20200913


In [7]:
def split_by_day(df, day, label):
    day_df = df[['origLocation', 'population', day]].copy()
    day_df.rename(columns={day: label}, inplace=True)
    day_df['date'] = day
    return day_df

## Admin1 divisions (Estados)

### Process Mexico confirmed cases

In [8]:
try:
    confirmed = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Estado_Nacional_Confirmados_{tomorrow}.csv", dtype='str')
    print("Loading data from:", tomorrow)
except:
    print("no data found for", tomorrow)

no data found for 20200913


In [9]:
try:
    confirmed = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Estado_Nacional_Confirmados_{today}.csv", dtype='str')
    print("Loading data from:", today)
except:
    print("no data found for", today)                      

no data found for 20200912


In [10]:
try:
    confirmed = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Estado_Nacional_Confirmados_{yesterday}.csv", dtype='str')
    print("Loading data from:", yesterday)
except:
    print("no data found for", yesterday)                         

Loading data from: 20200911


In [11]:
confirmed = confirmed.fillna('')

Remove total number of cases for the country

In [12]:
confirmed = confirmed[~(confirmed['nombre'] == 'Nacional')]

In [13]:
confirmed.head()

,cve_ent,poblacion,nombre,12-01-2020,13-01-2020,14-01-2020,15-01-2020,16-01-2020,17-01-2020,18-01-2020,19-01-2020,20-01-2020,21-01-2020,22-01-2020,23-01-2020,24-01-2020,25-01-2020,26-01-2020,27-01-2020,28-01-2020,29-01-2020,30-01-2020,31-01-2020,01-02-2020,02-02-2020,03-02-2020,04-02-2020,05-02-2020,06-02-2020,07-02-2020,08-02-2020,09-02-2020,10-02-2020,11-02-2020,12-02-2020,13-02-2020,14-02-2020,15-02-2020,16-02-2020,17-02-2020,18-02-2020,19-02-2020,20-02-2020,21-02-2020,22-02-2020,23-02-2020,24-02-2020,25-02-2020,26-02-2020,27-02-2020,28-02-2020,29-02-2020,01-03-2020,02-03-2020,03-03-2020,04-03-2020,05-03-2020,06-03-2020,07-03-2020,08-03-2020,09-03-2020,10-03-2020,11-03-2020,12-03-2020,13-03-2020,14-03-2020,15-03-2020,16-03-2020,17-03-2020,18-03-2020,19-03-2020,20-03-2020,21-03-2020,22-03-2020,23-03-2020,24-03-2020,25-03-2020,26-03-2020,27-03-2020,28-03-2020,29-03-2020,30-03-2020,31-03-2020,01-04-2020,02-04-2020,03-04-2020,04-04-2020,05-04-2020,06-04-2020,07-04-2020,08-04-2020,09-04-2020,10-04-2020,11-04-2020,12-04-2020,13-04-2020,14-04-2020,15-04-2020,16-04-2020,17-04-2020,18-04-2020,19-04-2020,20-04-2020,21-04-2020,22-04-2020,23-04-2020,24-04-2020,25-04-2020,26-04-2020,27-04-2020,28-04-2020,29-04-2020,30-04-2020,01-05-2020,02-05-2020,03-05-2020,04-05-2020,05-05-2020,06-05-2020,07-05-2020,08-05-2020,09-05-2020,10-05-2020,11-05-2020,12-05-2020,13-05-2020,14-05-2020,15-05-2020,16-05-2020,17-05-2020,18-05-2020,19-05-2020,20-05-2020,21-05-2020,22-05-2020,23-05-2020,24-05-2020,25-05-2020,26-05-2020,27-05-2020,28-05-2020,29-05-2020,30-05-2020,31-05-2020,01-06-2020,02-06-2020,03-06-2020,04-06-2020,05-06-2020,06-06-2020,07-06-2020,08-06-2020,09-06-2020,10-06-2020,11-06-2020,12-06-2020,13-06-2020,14-06-2020,15-06-2020,16-06-2020,17-06-2020,18-06-2020,19-06-2020,20-06-2020,21-06-2020,22-06-2020,23-06-2020,24-06-2020,25-06-2020,26-06-2020,27-06-2020,28-06-2020,29-06-2020,30-06-2020,01-07-2020,02-07-2020,03-07-2020,04-07-2020,05-07-2020,06-07-2020,07-07-2020,08-07-2020,09-07-2020,10-07-2020,11-07-2020,12-07-2020,13-07-2020,14-07-2020,15-07-2020,16-07-2020,17-07-2020,18-07-2020,19-07-2020,20-07-2020,21-07-2020,22-07-2020,23-07-2020,24-07-2020,25-07-2020,26-07-2020,27-07-2020,28-07-2020,29-07-2020,30-07-2020,31-07-2020,01-08-2020,02-08-2020,03-08-2020,04-08-2020,05-08-2020,06-08-2020,07-08-2020,08-08-2020,09-08-2020,10-08-2020,11-08-2020,12-08-2020,13-08-2020,14-08-2020,15-08-2020,16-08-2020,17-08-2020,18-08-2020,19-08-2020,20-08-2020,21-08-2020,22-08-2020,23-08-2020,24-08-2020,25-08-2020,26-08-2020,27-08-2020,28-08-2020,29-08-2020,30-08-2020,31-08-2020,01-09-2020,02-09-2020,03-09-2020,04-09-2020,05-09-2020,06-09-2020,07-09-2020,08-09-2020,09-09-2020,10-09-2020,11-09-2020
0,01,1434635,AGUASCALIENTES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,3,1,2,1,1,4,0,17,4,6,5,3,4,1,0,0,2,0,2,4,1,1,0,3,1,2,8,2,12,7,18,13,3,17,13,11,21,11,1,7,17,32,18,27,4,4,3,50,12,16,24,14,0,4,11,43,14,23,28,3,7,37,20,27,30,24,17,8,39,39,36,45,42,15,20,63,60,60,44,60,24,26,73,66,63,63,60,33,23,80,58,73,63,69,15,26,68,54,50,69,56,17,11,38,39,79,69,67,11,8,91,119,53,43,77,21,32,82,82,81,85,73,22,15,51,97,52,72,73,18,21,69,63,59,58,60,35,17,61,71,49,40,42,22,19,77,51,57,61,53,14,33,90,78,64,55,65,40,35,66,66,100,56,70,42,22,90,55,79,73,47,12,16,42,6,1,0,0
1,02,3634868,BAJA CALIFORNIA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,3,4,5,6,3,2,10,9,7,11,16,7,21,29,44,66,30,38,25,20,70,41,58,58,49,32,44,81,104,102,84,99,46,44,110,108,91,111,87,44,68,112,98,109,89,92,78,91,145,141,118,124,142,75,69,131,125,129,113,147,70,72,215,156,170,230,208,66,80,136,134,130,118,115,84,71,152,169,162,147,152,103,69,171,172,145,148,110,87,93,177,187,150,143,175,84,62,208,195,178,173,171,110,62,206,194,176,191,162,71,91,209,208,209,158,126,55,72,145,158,166,149,127,77,89,149,131,130,113,125,77,57,133,119,115,130,151,59,

In [14]:
confirmed['origLocation'] = 'Mexico,' + confirmed['nombre']

In [15]:
confirmed.rename(columns={'poblacion': 'population'}, inplace=True)

#### Reformat dataframe by day

In [16]:
days = confirmed.columns.tolist()[3:-1]

In [17]:
cases_confirmed = pd.concat((split_by_day(confirmed, day, 'cases') for day in days))
cases_confirmed['date'] = pd.to_datetime(cases_confirmed.date, format='%d-%m-%Y')

In [18]:
cases_confirmed.sort_values(['date', 'origLocation'], inplace=True)

In [19]:
cases_confirmed.head()

,origLocation,population,cases,date
0,"Mexico,AGUASCALIENTES",1434635,0,2020-01-12
1,"Mexico,BAJA CALIFORNIA",3634868,0,2020-01-12
2,"Mexico,BAJA CALIFORNIA SUR",804708,0,2020-01-12
3,"Mexico,CAMPECHE",1000617,0,2020-01-12
4,"Mexico,CHIAPAS",5730367,0,2020-01-12


### Process Mexico deaths

In [20]:
try:
    deaths = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Estado_Nacional_Defunciones_{tomorrow}.csv", dtype='str')
    print("Loading data from:", tomorrow)
except:
    print("no data found for", tomorrow)

no data found for 20200913


In [21]:
try:
    deaths = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Estado_Nacional_Defunciones_{today}.csv", dtype='str')
    print("Loading data from:", today)
except:
    print("no data found for", today)

no data found for 20200912


In [22]:
try:
    deaths = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Estado_Nacional_Defunciones_{yesterday}.csv", dtype='str')
    print("Loading data from:", yesterday)
except:
    print("no data found for", yesterday)

Loading data from: 20200911


In [23]:
deaths = deaths.fillna('')

Remove total number of cases for the country

In [24]:
deaths = deaths[~(deaths['nombre'] == 'Nacional')]

In [25]:
deaths.head()

,cve_ent,poblacion,nombre,17-03-2020,18-03-2020,19-03-2020,20-03-2020,21-03-2020,22-03-2020,23-03-2020,24-03-2020,25-03-2020,26-03-2020,27-03-2020,28-03-2020,29-03-2020,30-03-2020,31-03-2020,01-04-2020,02-04-2020,03-04-2020,04-04-2020,05-04-2020,06-04-2020,07-04-2020,08-04-2020,09-04-2020,10-04-2020,11-04-2020,12-04-2020,13-04-2020,14-04-2020,15-04-2020,16-04-2020,17-04-2020,18-04-2020,19-04-2020,20-04-2020,21-04-2020,22-04-2020,23-04-2020,24-04-2020,25-04-2020,26-04-2020,27-04-2020,28-04-2020,29-04-2020,30-04-2020,01-05-2020,02-05-2020,03-05-2020,04-05-2020,05-05-2020,06-05-2020,07-05-2020,08-05-2020,09-05-2020,10-05-2020,11-05-2020,12-05-2020,13-05-2020,14-05-2020,15-05-2020,16-05-2020,17-05-2020,18-05-2020,19-05-2020,20-05-2020,21-05-2020,22-05-2020,23-05-2020,24-05-2020,25-05-2020,26-05-2020,27-05-2020,28-05-2020,29-05-2020,30-05-2020,31-05-2020,01-06-2020,02-06-2020,03-06-2020,04-06-2020,05-06-2020,06-06-2020,07-06-2020,08-06-2020,09-06-2020,10-06-2020,11-06-2020,12-06-2020,13-06-2020,14-06-2020,15-06-2020,16-06-2020,17-06-2020,18-06-2020,19-06-2020,20-06-2020,21-06-2020,22-06-2020,23-06-2020,24-06-2020,25-06-2020,26-06-2020,27-06-2020,28-06-2020,29-06-2020,30-06-2020,01-07-2020,02-07-2020,03-07-2020,04-07-2020,05-07-2020,06-07-2020,07-07-2020,08-07-2020,09-07-2020,10-07-2020,11-07-2020,12-07-2020,13-07-2020,14-07-2020,15-07-2020,16-07-2020,17-07-2020,18-07-2020,19-07-2020,20-07-2020,21-07-2020,22-07-2020,23-07-2020,24-07-2020,25-07-2020,26-07-2020,27-07-2020,28-07-2020,29-07-2020,30-07-2020,31-07-2020,01-08-2020,02-08-2020,03-08-2020,04-08-2020,05-08-2020,06-08-2020,07-08-2020,08-08-2020,09-08-2020,10-08-2020,11-08-2020,12-08-2020,13-08-2020,14-08-2020,15-08-2020,16-08-2020,17-08-2020,18-08-2020,19-08-2020,20-08-2020,21-08-2020,22-08-2020,23-08-2020,24-08-2020,25-08-2020,26-08-2020,27-08-2020,28-08-2020,29-08-2020,30-08-2020,31-08-2020,01-09-2020,02-09-2020,03-09-2020,04-09-2020,05-09-2020,06-09-2020,07-09-2020,08-09-2020,09-09-2020,10-09-2020
0,01,1434635,AGUASCALIENTES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,3,3,1,0,1,0,0,2,1,1,1,4,0,0,0,2,1,0,4,1,1,4,3,3,2,2,2,3,4,4,1,5,4,2,4,6,3,7,2,3,3,5,4,6,5,4,5,1,2,6,3,10,4,1,4,2,2,3,4,8,4,2,6,6,3,1,4,4,3,5,3,4,2,3,2,4,2,2,8,8,5,4,3,3,2,3,4,3,2,2,6,2,6,7,5,5,3,3,5,5,5,8,4,4,5,4,5,4,2,2,13,6,5,10,6,4,1,8,4,13,6,9,7,3,2,3,3,0
1,02,3634868,BAJA CALIFORNIA,0,0,0,0,0,0,0,0,0,0,0,0,1,2,2,2,5,2,5,1,10,5,6,5,7,10,12,8,12,17,11,20,16,16,14,15,18,19,24,18,18,28,21,29,22,23,26,14,30,31,28,26,34,36,28,31,30,38,36,27,30,40,52,33,41,21,26,27,32,32,30,29,37,18,33,29,32,29,25,28,26,39,28,30,18,29,22,32,27,30,22,18,30,24,22,13,19,23,24,24,25,25,20,16,28,31,22,25,27,27,33,44,27,23,21,27,24,23,17,18,8,23,26,17,24,25,17,20,22,16,27,16,22,20,15,22,15,14,15,21,29,13,10,13,19,7,16,21,6,11,18,12,16,27,18,12,11,13,14,13,16,11,5,7,15,8,20,13,15,14,6,8,11,5,12,5,9,1
2,03,804708,BAJA CALIFORNIA SUR,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,0,1,1,1,0,0,1,1,0,0,1,1,2,0,1,0,2,0,1,1,0,0,0,1,1,0,2,0,2,0,2,1,0,1,1,0,1,1,1,0,1,0,1,0,1,0,1,2,0,1,0,0,1,1,2,3,3,0,0,0,1,0,2,2,2,4,4,1,1,1,1,2,0,1,1,1,1,1,3,0,1,4,1,5,4,1,2,2,0,1,3,3,4,5,4,4,3,2,7,2,3,4,4,5,4,7,3,7,5,6,8,8,3,6,4,3,4,4,1,6,8,8,5,12,6,3,9,5,7,7,12,8,6,6,4,5,6,8,3,3,3,5,3,8,4,4,6,2,1,2,0,1,5,0
3,04,1000617,CAMPECHE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,2,0,1,2,2,2,2,1,1,0,3,0,2,2,1,0,1,1,1,1,3,1,2,2,1,4,5,3,2,2,2,4,1,1,1,5,3,3,1,4,3,2,0,2,4,3,2,2,2,6,9,3,2,2,2,7,5,6,3,8,2,4,9,4,5,10,8,5,9,15,13,10,6,12,8,12,9,7,12,16,11,13,17,13,10,11,10,7,15,9,9,14,9,14,8,7,9,10,15,9,8,13,4,10,12,13,17,7,8,12,4,6,9,5,7,5,7,7,9,3,6,4,4,1,3,4,4,2,1,4,1,0,2,11,3,3,2,0,2,2,1,0,1,0,1
4,07,5730367,CHIAPAS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,1,1,0,0,0,1,1,1,1,1,4,0,0,3,1,1,2,4,7,5,5,5,3,2,3,6,5,11,6,2,10,8,11,13,4,11,13,12,14,11,15,25,22,16,19,10,24,26,26,15,11,23,13,12,15,21,14,15,11,12,10,17,17,22,12,16,13,26,11,18,11,8,12,15,18,9,18,11,12,14,13,16,1

In [26]:
deaths['origLocation'] = 'Mexico,' + deaths['nombre']

In [27]:
deaths.rename(columns={'poblacion': 'population'}, inplace=True)

#### Reformat dataframe by day

In [28]:
days = deaths.columns.tolist()[3:-1]

In [29]:
cases_deaths = pd.concat((split_by_day(deaths, day, 'deaths') for day in days))
#cases_deaths['date'] = cases_deaths['date'].apply(dateutil.parser.parse)

In [30]:
cases_deaths['date'] = pd.to_datetime(cases_deaths['date'], format='%d-%m-%Y')

In [31]:
cases_deaths.sort_values(['date', 'origLocation'], inplace=True)

In [32]:
cases_deaths.head()

,origLocation,population,deaths,date
0,"Mexico,AGUASCALIENTES",1434635,0,2020-03-17
1,"Mexico,BAJA CALIFORNIA",3634868,0,2020-03-17
2,"Mexico,BAJA CALIFORNIA SUR",804708,0,2020-03-17
3,"Mexico,CAMPECHE",1000617,0,2020-03-17
4,"Mexico,CHIAPAS",5730367,0,2020-03-17


### Merge US cases into one dataframe

In [33]:
cases = cases_confirmed.merge(cases_deaths, on=['origLocation', 'population', 'date'], how="left")
cases = cases.fillna('0')

In [34]:
cases.head()

,origLocation,population,cases,date,deaths
0,"Mexico,AGUASCALIENTES",1434635,0,2020-01-12,0
1,"Mexico,BAJA CALIFORNIA",3634868,0,2020-01-12,0
2,"Mexico,BAJA CALIFORNIA SUR",804708,0,2020-01-12,0
3,"Mexico,CAMPECHE",1000617,0,2020-01-12,0
4,"Mexico,CHIAPAS",5730367,0,2020-01-12,0


In [35]:
cases['cases'] = cases['cases'].astype(int)
cases['deaths'] = cases['deaths'].astype(int)

In [36]:
cases.dtypes

origLocation            object
population              object
cases                    int64
date            datetime64[ns]
deaths                   int64
dtype: object

In [37]:
cases['cumulativeCases'] = cases.groupby(['origLocation'])['cases'].cumsum()
cases['cumulativeDeaths'] = cases.groupby(['origLocation'])['deaths'].cumsum()

In [38]:
cases.query("origLocation == 'Mexico,BAJA CALIFORNIA'").tail(10)

,origLocation,population,cases,date,deaths,cumulativeCases,cumulativeDeaths
7489,"Mexico,BAJA CALIFORNIA",3634868,149,2020-09-02,14,17799,3274
7521,"Mexico,BAJA CALIFORNIA",3634868,141,2020-09-03,6,17940,3280
7553,"Mexico,BAJA CALIFORNIA",3634868,99,2020-09-04,8,18039,3288
7585,"Mexico,BAJA CALIFORNIA",3634868,30,2020-09-05,11,18069,3299
7617,"Mexico,BAJA CALIFORNIA",3634868,12,2020-09-06,5,18081,3304
7649,"Mexico,BAJA CALIFORNIA",3634868,48,2020-09-07,12,18129,3316
7681,"Mexico,BAJA CALIFORNIA",3634868,19,2020-09-08,5,18148,3321
7713,"Mexico,BAJA CALIFORNIA",3634868,9,2020-09-09,9,18157,3330
7745,"Mexico,BAJA CALIFORNIA",3634868,0,2020-09-10,1,18157,3331
7777,"Mexico,BAJA CALIFORNIA",3634868,0,2020-09-11,0,18157,3331


In [39]:
cases.to_csv(NEO4J_IMPORT / "02d-GOBMXCasesAdmin1.csv", index=False)

## Admin2 divisions (Municipios)

### Process Mexico confirmed cases

In [40]:
try:
    confirmed = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Municipio_Confirmados_{tomorrow}.csv",  dtype='str')
    print("Loading data from:", tomorrow)
except:
    print("no data found for", tomorrow)

no data found for 20200913


In [41]:
try:
    confirmed = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Municipio_Confirmados_{today}.csv",  dtype='str')
    print("Loading data from:", today)
except:
    print("no data found for", today)

no data found for 20200912


In [42]:
try:
    confirmed = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Municipio_Confirmados_{yesterday}.csv",  dtype='str')
    print("Loading data from:", yesterday)
except:
    print("no data found for", yesterday)

Loading data from: 20200911


In [43]:
confirmed = confirmed.fillna('')

Remove total number of cases for the country

In [44]:
confirmed = confirmed[~(confirmed['nombre'] == 'Nacional')]

In [45]:
confirmed.head()

,cve_ent,poblacion,nombre,12-01-2020,13-01-2020,14-01-2020,15-01-2020,16-01-2020,17-01-2020,18-01-2020,19-01-2020,20-01-2020,21-01-2020,22-01-2020,23-01-2020,24-01-2020,25-01-2020,26-01-2020,27-01-2020,28-01-2020,29-01-2020,30-01-2020,31-01-2020,01-02-2020,02-02-2020,03-02-2020,04-02-2020,05-02-2020,06-02-2020,07-02-2020,08-02-2020,09-02-2020,10-02-2020,11-02-2020,12-02-2020,13-02-2020,14-02-2020,15-02-2020,16-02-2020,17-02-2020,18-02-2020,19-02-2020,20-02-2020,21-02-2020,22-02-2020,23-02-2020,24-02-2020,25-02-2020,26-02-2020,27-02-2020,28-02-2020,29-02-2020,01-03-2020,02-03-2020,03-03-2020,04-03-2020,05-03-2020,06-03-2020,07-03-2020,08-03-2020,09-03-2020,10-03-2020,11-03-2020,12-03-2020,13-03-2020,14-03-2020,15-03-2020,16-03-2020,17-03-2020,18-03-2020,19-03-2020,20-03-2020,21-03-2020,22-03-2020,23-03-2020,24-03-2020,25-03-2020,26-03-2020,27-03-2020,28-03-2020,29-03-2020,30-03-2020,31-03-2020,01-04-2020,02-04-2020,03-04-2020,04-04-2020,05-04-2020,06-04-2020,07-04-2020,08-04-2020,09-04-2020,10-04-2020,11-04-2020,12-04-2020,13-04-2020,14-04-2020,15-04-2020,16-04-2020,17-04-2020,18-04-2020,19-04-2020,20-04-2020,21-04-2020,22-04-2020,23-04-2020,24-04-2020,25-04-2020,26-04-2020,27-04-2020,28-04-2020,29-04-2020,30-04-2020,01-05-2020,02-05-2020,03-05-2020,04-05-2020,05-05-2020,06-05-2020,07-05-2020,08-05-2020,09-05-2020,10-05-2020,11-05-2020,12-05-2020,13-05-2020,14-05-2020,15-05-2020,16-05-2020,17-05-2020,18-05-2020,19-05-2020,20-05-2020,21-05-2020,22-05-2020,23-05-2020,24-05-2020,25-05-2020,26-05-2020,27-05-2020,28-05-2020,29-05-2020,30-05-2020,31-05-2020,01-06-2020,02-06-2020,03-06-2020,04-06-2020,05-06-2020,06-06-2020,07-06-2020,08-06-2020,09-06-2020,10-06-2020,11-06-2020,12-06-2020,13-06-2020,14-06-2020,15-06-2020,16-06-2020,17-06-2020,18-06-2020,19-06-2020,20-06-2020,21-06-2020,22-06-2020,23-06-2020,24-06-2020,25-06-2020,26-06-2020,27-06-2020,28-06-2020,29-06-2020,30-06-2020,01-07-2020,02-07-2020,03-07-2020,04-07-2020,05-07-2020,06-07-2020,07-07-2020,08-07-2020,09-07-2020,10-07-2020,11-07-2020,12-07-2020,13-07-2020,14-07-2020,15-07-2020,16-07-2020,17-07-2020,18-07-2020,19-07-2020,20-07-2020,21-07-2020,22-07-2020,23-07-2020,24-07-2020,25-07-2020,26-07-2020,27-07-2020,28-07-2020,29-07-2020,30-07-2020,31-07-2020,01-08-2020,02-08-2020,03-08-2020,04-08-2020,05-08-2020,06-08-2020,07-08-2020,08-08-2020,09-08-2020,10-08-2020,11-08-2020,12-08-2020,13-08-2020,14-08-2020,15-08-2020,16-08-2020,17-08-2020,18-08-2020,19-08-2020,20-08-2020,21-08-2020,22-08-2020,23-08-2020,24-08-2020,25-08-2020,26-08-2020,27-08-2020,28-08-2020,29-08-2020,30-08-2020,31-08-2020,01-09-2020,02-09-2020,03-09-2020,04-09-2020,05-09-2020,06-09-2020,07-09-2020,08-09-2020,09-09-2020,10-09-2020,11-09-2020
0,16046,15290,Juarez,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,1,1,1,0,0,0,2,0,0,5,0,0,0,2,0,0,0,3,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,1,0,0,0,1,0,2,0,5,3,0,1,1,2,1,3,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,3,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
1,16047,22358,Jungapeo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,2,0,0,0,0,1,2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0
2,16048,5862,Lagunillas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [46]:
confirmed['origLocation'] = 'Mexico,' + confirmed['nombre']

In [47]:
confirmed.rename(columns={'poblacion': 'population'}, inplace=True)

#### Reformat dataframe by day

In [48]:
days = confirmed.columns.tolist()[3:-1]

In [49]:
cases_confirmed = pd.concat((split_by_day(confirmed, day, 'cases') for day in days))
cases_confirmed['date'] = pd.to_datetime(cases_confirmed.date, format='%d-%m-%Y')

In [50]:
cases_confirmed.sort_values(['date', 'origLocation'], inplace=True)

In [51]:
cases_confirmed.head()

,origLocation,population,cases,date
2349,"Mexico,Abala",7035,0,2020-01-12
486,"Mexico,Abasolo",1130,0,2020-01-12
1080,"Mexico,Abasolo",95581,0,2020-01-12
1353,"Mexico,Abasolo",2918,0,2020-01-12
2157,"Mexico,Abasolo",12768,0,2020-01-12


### Process Mexico deaths

In [52]:
try:
    deaths = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Municipio_Defunciones_{tomorrow}.csv", dtype='str')
    print("Loading data from:", tomorrow)
except:
    print("no data found for", tomorrow)

no data found for 20200913


In [53]:
try:
    deaths = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Municipio_Defunciones_{today}.csv", dtype='str')
    print("Loading data from:", today)
except:
    print("no data found for", today)

no data found for 20200912


In [54]:
try:
    deaths = pd.read_csv(f"https://coronavirus.gob.mx/datos/Downloads/Files/Casos_Diarios_Municipio_Defunciones_{yesterday}.csv", dtype='str')
    print("Loading data from:", yesterday)
except:
    print("no data found for", yesterday)

Loading data from: 20200911


In [55]:
deaths = deaths.fillna('')

Remove total number of cases for the country

In [56]:
deaths = deaths[~(deaths['nombre'] == 'Nacional')]

In [57]:
deaths.head()

,cve_ent,poblacion,nombre,17-03-2020,18-03-2020,19-03-2020,20-03-2020,21-03-2020,22-03-2020,23-03-2020,24-03-2020,25-03-2020,26-03-2020,27-03-2020,28-03-2020,29-03-2020,30-03-2020,31-03-2020,01-04-2020,02-04-2020,03-04-2020,04-04-2020,05-04-2020,06-04-2020,07-04-2020,08-04-2020,09-04-2020,10-04-2020,11-04-2020,12-04-2020,13-04-2020,14-04-2020,15-04-2020,16-04-2020,17-04-2020,18-04-2020,19-04-2020,20-04-2020,21-04-2020,22-04-2020,23-04-2020,24-04-2020,25-04-2020,26-04-2020,27-04-2020,28-04-2020,29-04-2020,30-04-2020,01-05-2020,02-05-2020,03-05-2020,04-05-2020,05-05-2020,06-05-2020,07-05-2020,08-05-2020,09-05-2020,10-05-2020,11-05-2020,12-05-2020,13-05-2020,14-05-2020,15-05-2020,16-05-2020,17-05-2020,18-05-2020,19-05-2020,20-05-2020,21-05-2020,22-05-2020,23-05-2020,24-05-2020,25-05-2020,26-05-2020,27-05-2020,28-05-2020,29-05-2020,30-05-2020,31-05-2020,01-06-2020,02-06-2020,03-06-2020,04-06-2020,05-06-2020,06-06-2020,07-06-2020,08-06-2020,09-06-2020,10-06-2020,11-06-2020,12-06-2020,13-06-2020,14-06-2020,15-06-2020,16-06-2020,17-06-2020,18-06-2020,19-06-2020,20-06-2020,21-06-2020,22-06-2020,23-06-2020,24-06-2020,25-06-2020,26-06-2020,27-06-2020,28-06-2020,29-06-2020,30-06-2020,01-07-2020,02-07-2020,03-07-2020,04-07-2020,05-07-2020,06-07-2020,07-07-2020,08-07-2020,09-07-2020,10-07-2020,11-07-2020,12-07-2020,13-07-2020,14-07-2020,15-07-2020,16-07-2020,17-07-2020,18-07-2020,19-07-2020,20-07-2020,21-07-2020,22-07-2020,23-07-2020,24-07-2020,25-07-2020,26-07-2020,27-07-2020,28-07-2020,29-07-2020,30-07-2020,31-07-2020,01-08-2020,02-08-2020,03-08-2020,04-08-2020,05-08-2020,06-08-2020,07-08-2020,08-08-2020,09-08-2020,10-08-2020,11-08-2020,12-08-2020,13-08-2020,14-08-2020,15-08-2020,16-08-2020,17-08-2020,18-08-2020,19-08-2020,20-08-2020,21-08-2020,22-08-2020,23-08-2020,24-08-2020,25-08-2020,26-08-2020,27-08-2020,28-08-2020,29-08-2020,30-08-2020,31-08-2020,01-09-2020,02-09-2020,03-09-2020,04-09-2020,05-09-2020,06-09-2020,07-09-2020,08-09-2020,09-09-2020,10-09-2020
0,16046,15290,Juarez,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,16047,22358,Jungapeo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,16048,5862,Lagunillas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,16049,18769,Madero,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,05035,744247,Torreon,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,1,1,1,0,1,2,0,2,1,0,0,1,0,3,0,5,1,4,4,4,2,3,3,3,2,1,3,8,0,0,0,7,4,1,4,2,4,5,5,8,6,4,2,6,4,2,4,4,4,3,3,2,0,2,4,7,4,2,0,5,8,10,4,3,1,3,3,3,6,3,5,4,7,6,4,6,5,2,7,4,4,2,3,3,2,2,4,5,1,1,3,1,2,4,5,3,0,1,2,0,0,1,5,1,0,4,2,3,3,2,4,2,1


In [58]:
deaths['origLocation'] = 'Mexico,' + deaths['nombre']

In [59]:
deaths.rename(columns={'poblacion': 'population'}, inplace=True)

#### Reformat dataframe by day

In [60]:
days = deaths.columns.tolist()[3:-1]

In [61]:
cases_deaths = pd.concat((split_by_day(deaths, day, 'deaths') for day in days))

In [62]:
cases_deaths['date'] = pd.to_datetime(cases_deaths['date'], format='%d-%m-%Y')

In [63]:
cases_deaths.sort_values(['date', 'origLocation'], inplace=True)

In [64]:
cases_deaths.head()

,origLocation,population,deaths,date
2349,"Mexico,Abala",7035,0,2020-03-17
486,"Mexico,Abasolo",1130,0,2020-03-17
1080,"Mexico,Abasolo",95581,0,2020-03-17
1353,"Mexico,Abasolo",2918,0,2020-03-17
2157,"Mexico,Abasolo",12768,0,2020-03-17


### Merge US cases into one dataframe

In [65]:
cases = cases_confirmed.merge(cases_deaths, on=['origLocation', 'population', 'date'], how="left")
cases = cases.fillna('0')

In [66]:
cases.head()

,origLocation,population,cases,date,deaths
0,"Mexico,Abala",7035,0,2020-01-12,0
1,"Mexico,Abasolo",1130,0,2020-01-12,0
2,"Mexico,Abasolo",95581,0,2020-01-12,0
3,"Mexico,Abasolo",2918,0,2020-01-12,0
4,"Mexico,Abasolo",12768,0,2020-01-12,0


In [67]:
cases['cases'] = cases['cases'].astype(int)
cases['deaths'] = cases['deaths'].astype(int)

In [68]:
cases['cumulativeCases'] = cases.groupby(['origLocation'])['cases'].cumsum()
cases['cumulativeDeaths'] = cases.groupby(['origLocation'])['deaths'].cumsum()

In [69]:
cases.query("origLocation == 'Mexico,Tijuana'").tail(10)

,origLocation,population,cases,date,deaths,cumulativeCases,cumulativeDeaths
577077,"Mexico,Tijuana",1789531,65,2020-09-02,6,5603,1300
579534,"Mexico,Tijuana",1789531,62,2020-09-03,3,5665,1303
581991,"Mexico,Tijuana",1789531,45,2020-09-04,3,5710,1306
584448,"Mexico,Tijuana",1789531,16,2020-09-05,7,5726,1313
586905,"Mexico,Tijuana",1789531,2,2020-09-06,3,5728,1316
589362,"Mexico,Tijuana",1789531,12,2020-09-07,4,5740,1320
591819,"Mexico,Tijuana",1789531,0,2020-09-08,3,5740,1323
594276,"Mexico,Tijuana",1789531,1,2020-09-09,3,5741,1326
596733,"Mexico,Tijuana",1789531,0,2020-09-10,0,5741,1326
599190,"Mexico,Tijuana",1789531,0,2020-09-11,0,5741,1326


In [70]:
cases.to_csv(NEO4J_IMPORT / "02d-GOBMXCasesAdmin2.csv", index=False)